In [90]:
import pandas as pd

# 파일 경로
file_school = "./merge/유치원_초등학교_행정동코드_매핑완료.csv"
file_nursery = "./merge/서울시_어린이집_행정동코드_최종완료.csv"

# 파일 불러오기
df_school = pd.read_csv(file_school)
df_nursery = pd.read_csv(file_nursery)

# 초등학교/유치원 분리
df_elem = df_school[df_school['시설유형'] == '초등학교'].copy()
df_kinder = df_school[df_school['시설유형'] == '유치원'].copy()

# 초등학교: 위도, 경도 컬럼 유지
df_elem_full = df_school[df_school['시설유형'] == '초등학교'].copy()
df_elem_full = df_elem_full.rename(columns={
    '시설명': '시설명',
    '도로명 주소': '시설주소',
    '행자부행정동코드': '행자부행정동코드',
    '위도': '위도',
    '경도': '경도'
})[['시설명', '시설주소', '행자부행정동코드', '위도', '경도']]

# 유치원: 위도, 경도 컬럼 유지
df_kinder_fixed = df_school[df_school['시설유형'].str.contains('유치원')].copy()

df_kinder_fixed = df_kinder_fixed.rename(columns={
    '시설명': '시설명',
    '도로명 주소': '시설주소',
    '행자부행정동코드': '행자부행정동코드',
    '위도': '위도',
    '경도': '경도'
})[['시설명', '시설주소', '행자부행정동코드', '위도', '경도']]

# 어린이집: 위도, 경도 컬럼도 변경
df_nursery_full = df_nursery.copy()
df_nursery_full = df_nursery.rename(columns={
    '어린이집명': '시설명',
    '상세주소': '시설주소',
    '행자부행정동코드': '행자부행정동코드',
    '시설 위도(좌표값)_수정': '위도',
    '시설 경도(좌표값)_수정': '경도'
})[['시설명', '시설주소', '행자부행정동코드', '위도', '경도']]

# 유치원 중 어린이집과 중복 제거
df_kinder_filtered = df_kinder_fixed.merge(
    df_nursery_full, on=['시설명', '행자부행정동코드'], how='left', indicator=True
)
df_kinder_filtered = df_kinder_filtered[df_kinder_filtered['_merge'] == 'left_only']
df_kinder_filtered = df_kinder_filtered.drop(columns=['_merge', '시설주소_y', '위도_y', '경도_y']).rename(columns={
    '시설주소_x': '시설주소',
    '위도_x': '위도',
    '경도_x': '경도'
})

# 최종 병합
df_merged_with_kinder = pd.merge(df_elem_full, df_kinder_filtered, how='outer')
df_merged_with_kinder = pd.merge(df_merged_with_kinder, df_nursery_full, how='outer')

### 없는거 채우기

In [94]:
import numpy as np

print(df_merged_with_kinder[df_merged_with_kinder['행자부행정동코드'].isna()==True])

mapping = {'강동유정유치원':1174052600,
           '고덕유치원':1174052500,
           '선아유치원':1174052500,
           '연유치원':1141070000,
           '이화유치원':1168067500,
           '충암유치원':1138060000,
           '한영중고등학교병설한영유치원':1174052500,
           '서울강명초등학교':1174052600,
'서울고일초등학교':1174052500,
'서울고현초등학교':1174052500,
'서울대진초등학교':1168067500,
'서울면중초등학교':1126057500,
'서울상일초등학교':1174052500,
'서울항동초등학교':1153080000}

df_merged_with_kinder['행자부행정동코드'] = df_merged_with_kinder.apply(
    lambda row: mapping[row['시설명']] if pd.isna(row['행자부행정동코드']) and row['시설명'] in mapping else row['행자부행정동코드'],
    axis=1
)

print(df_merged_with_kinder[df_merged_with_kinder['행자부행정동코드'].isna()==True])

                 시설명        시설주소  행자부행정동코드         위도          경도
141          강동유정유치원   상일로10길 41       NaN  37.550562  127.174954
194            고덕유치원    구천면로 645       NaN  37.551112  127.169246
2420        서울강명초등학교    상일로 74-1       NaN  37.554706  127.173374
2458        서울고일초등학교  구천면로93길 20       NaN  37.551543  127.169410
2461        서울고현초등학교  상일로11길 110       NaN  37.553804  127.166050
2561        서울대진초등학교  개포로109길 74       NaN  37.497028  127.077861
2629        서울면중초등학교  용마산로70길 43       NaN  37.582442  127.096551
2728        서울상일초등학교   천호대로 1477       NaN  37.547572  127.172663
3160        서울항동초등학교     연동로 178       NaN  37.477092  126.824190
3297           선아유치원    구천면로 650       NaN  37.550319  127.169808
3924            연유치원   명지대길 39-4       NaN  37.581879  126.924076
4398           이화유치원   개포로109길 5       NaN  37.492322  127.074601
4729           충암유치원     가좌로5길 5       NaN  37.584640  126.921809
5110  한영중고등학교병설한영유치원     동남로 832       NaN  37.549084  127.156928
Empty Data

In [98]:
df_merged_with_kinder.isna().sum()
df_merged_with_kinder.shape

(5409, 5)

In [100]:
df_merged_with_kinder.duplicated(['시설명', '행자부행정동코드']).sum()

1

In [102]:
df_merged_full_cleaned = df_merged_with_kinder.drop_duplicates(subset=['시설명', '행자부행정동코드'])

In [104]:
df_merged_full_cleaned.duplicated(['시설명', '행자부행정동코드']).sum()

0

In [116]:
df_merged_full_cleaned.duplicated(['위도', '경도']).sum()

1018

In [108]:
duplicate_latlng_rows = df_merged_full_cleaned[df_merged_full_cleaned.duplicated(['위도', '경도'], keep=False)]
duplicate_latlng_rows.sort_values(by=['위도', '경도'], inplace=True)
duplicate_latlng_rows

duplicate_latlng_rows.to_csv("./중복_위경도_시설목록.csv", index=False, encoding='utf-8-sig')

C:\Users\user\AppData\Local\Temp\ipykernel_10776\296533711.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  duplicate_latlng_rows.sort_values(by=['위도', '경도'], inplace=True)


### 신사동이 중복되어 분리해주기

In [114]:
duplicated_names = df_merged_full_cleaned[
    df_merged_full_cleaned['행자부행정동코드'].isin([11680510, 11620685])
]

# 중복된 시설명 중복 여부 확인
duplicated_facility_names = duplicated_names['시설명'].value_counts()
duplicated_facility_names = duplicated_facility_names[duplicated_facility_names > 1].index.tolist()

# 필터링
target_rows = df_merged_full_cleaned[
    (df_merged_full_cleaned['시설명'].isin(duplicated_facility_names)) &
    (df_merged_full_cleaned['행자부행정동코드'].isin([11680510, 11620685]))
].copy()

# 조건 적용: 주소에 '관악구'가 들어가면 -> 11620685만 남김
#            주소에 '강남구'나 '압구정동'이 들어가면 -> 11680510만 남김
#            그 외는 유지
mask_관악구 = (target_rows['시설주소'].str.contains('관악구')) & (target_rows['행자부행정동코드'] == 11680510)
mask_강남구압구정 = (target_rows['시설주소'].str.contains('강남구|압구정동')) & (target_rows['행자부행정동코드'] == 11620685)

# 삭제 대상 인덱스 모으기
drop_index = target_rows[mask_관악구 | mask_강남구압구정].index

# 원본 데이터에서 삭제
df_final_filtered = df_merged_full_cleaned.drop(index=drop_index)

In [118]:
df_final_filtered.duplicated(['위도', '경도']).sum()

1007

In [120]:
df_final_filtered.to_csv('./초등학교_유치원_어린이집_머지데이터.csv', index=False, encoding='utf-8=sig')

### 재배열

In [132]:
# 데이터 다시 로드
data = pd.read_csv('./초등학교_유치원_어린이집_머지데이터.csv')

# 시설유형 구분
data['시설유형'] = data['시설명'].apply(
    lambda x: '초등학교' if '초등학교' in x else ('어린이집' if '어린이집' in x else ('유치원' if '유치원' in x else '기타'))
)

# 시설유형별로 데이터프레임 분리 및 컬럼명 변경
school_df = data[data['시설유형'] == '초등학교'][['행자부행정동코드', '시설명', '시설주소', '위도', '경도']].rename(columns={
    '시설명': '시설명_초등학교', '시설주소': '시설주소_초등학교', '위도': '위도_초등학교', '경도': '경도_초등학교'})

childcare_df = data[data['시설유형'] == '어린이집'][['행자부행정동코드', '시설명', '시설주소', '위도', '경도']].rename(columns={
    '시설명': '시설명_어린이집', '시설주소': '시설주소_어린이집', '위도': '위도_어린이집', '경도': '경도_어린이집'})

kindergarten_df = data[data['시설유형'] == '유치원'][['행자부행정동코드', '시설명', '시설주소', '위도', '경도']].rename(columns={
    '시설명': '시설명_유치원', '시설주소': '시설주소_유치원', '위도': '위도_유치원', '경도': '경도_유치원'})

# 세 데이터프레임을 행자부행정동코드를 기준으로 병합
merged_df = pd.merge(school_df, childcare_df, on='행자부행정동코드', how='outer')
merged_df = pd.merge(merged_df, kindergarten_df, on='행자부행정동코드', how='outer')

# 최종 결과에서 중복 제거 및 결측값은 빈칸으로 채움
merged_df = merged_df.fillna('')
# 행자부행정동코드를 정수형으로 변환
merged_df['행자부행정동코드'] = merged_df['행자부행정동코드'].astype(int)

merged_df.head()

,행자부행정동코드,시설명_초등학교,시설주소_초등학교,위도_초등학교,경도_초등학교,시설명_어린이집,시설주소_어린이집,위도_어린이집,경도_어린이집,시설명_유치원,시설주소_유치원,위도_유치원,경도_유치원
0,11110515,서울청운초등학교,자하문로 105,37.585806,126.969295,누상어린이집,서울특별시 종로구 옥인길 23-4 (누상동),37.580793,126.967194,배화여자대학교 부속 배화유치원,필운대로1길 34,37.579544,126.967292
1,11110515,서울청운초등학교,자하문로 105,37.585806,126.969295,누상어린이집,서울특별시 종로구 옥인길 23-4 (누상동),37.580793,126.967194,옥인유치원,자하문로 69,37.582222,126.970343
2,11110515,서울청운초등학교,자하문로 105,37.585806,126.969295,무궁화어린이집,서울특별시 종로구 자하문로28길 8 외 1 (궁정동),37.584797,126.971101,배화여자대학교 부속 배화유치원,필운대로1길 34,37.579544,126.967292
3,11110515,서울청운초등학교,자하문로 105,37.585806,126.969295,무궁화어린이집,서울특별시 종로구 자하문로28길 8 외 1 (궁정동),37.584797,126.971101,옥인유치원,자하문로 69,37.582222,126.970343
4,11110515,서울청운초등학교,자하문로 105,37.585806,126.969295,세종마을어린이집,서울특별시 종로구 옥인길 89 (옥인동),37.581968,126.963941,배화여자대학교 부속 배화유치원,필운대로1길 34,37.579544,126.967292


In [134]:
merged_df.to_csv('./초등학교_유치원_어린이집_병합.csv', index=False, encoding='utf-8-sig')

In [138]:
merged_df.duplicated().sum()

0

### 리스트 형태로 변환

In [117]:
import pandas as pd
import requests
import time

# -----------------------------------
# SGIS API 함수
# -----------------------------------
def get_sgis_access_token(consumer_key: str, consumer_secret: str) -> str:
    url = "https://sgisapi.kostat.go.kr/OpenAPI3/auth/authentication.json"
    params = {"consumer_key": consumer_key, "consumer_secret": consumer_secret}
    response = requests.get(url, params=params)
    response.raise_for_status()
    return response.json()["result"]["accessToken"]

def get_adm_cd_from_address(address: str, access_token: str) -> str:
    url = "https://sgisapi.kostat.go.kr/OpenAPI3/addr/geocode.json"
    params = {"accessToken": access_token, "address": address}
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()

    if "result" in data and "resultdata" in data["result"] and data["result"]["resultdata"]:
        return data["result"]["resultdata"][0].get("adm_cd", "")
    return ""


# 1데이터 로드
df = pd.read_csv("./초등학교_유치원_어린이집_병합.csv")

# grouped 정의 (전체 병합 데이터 기준으로 먼저 구성)
grouped = df[["행자부행정동코드", "시설명_초등학교", "시설명_유치원", "시설명_어린이집"]].fillna("")

grouped = grouped.groupby("행자부행정동코드").agg({
    "시설명_초등학교": lambda x: list(set(i for i in x if i)),
    "시설명_유치원": lambda x: list(set(i for i in x if i)),
    "시설명_어린이집": lambda x: list(set(i for i in x if i)),
}).reset_index()

grouped.columns = ["행정동코드", "초등학교", "유치원", "어린이집"]

grouped["행정동코드"] = grouped["행정동코드"].astype(str).str[:8].astype(int)


# 2수유동 / 번동 주소만 추출
target_df = df[df["시설주소_어린이집"].str.contains("수유동|번동", na=False)].copy()


# SGIS API 사용: 주소 → 행정동코드 매핑
consumer_key = "e73744787c154c9db103"
consumer_secret = "6e49cc5f5f474ca6867c"
token = get_sgis_access_token(consumer_key, consumer_secret)

adm_codes = []
for addr in target_df["시설주소_어린이집"]:
    adm_cd = get_adm_cd_from_address(addr, token)
    adm_codes.append(adm_cd)
    time.sleep(1.0)

target_df["행정동코드"] = [int(code[:8]) if code else None for code in adm_codes]
target_df = target_df.dropna(subset=["행정동코드"]).copy()


# 필요한 컬럼 정리
target_df["초등학교"] = target_df["시설명_초등학교"].apply(lambda x: [f'{x}'] if pd.notna(x) else [])
target_df["유치원"] = target_df["시설명_유치원"].apply(lambda x: [f'{x}'] if pd.notna(x) else [])
target_df["어린이집"] = target_df["시설명_어린이집"].apply(lambda x: [f'{x}'] if pd.notna(x) else [])
target_df = target_df[["행정동코드", "초등학교", "유치원", "어린이집"]]


# 병합
# 상위코드 리스트 제거 대상
상위코드_제거 = [11305600, 11305606, 11305610, 11305620, 11305630, 11305590]

# 기존 리스트형 grouped에서 상위코드 제거
grouped_filtered = grouped[~grouped["행정동코드"].isin(상위코드_제거)].copy()

# 컬럼들 문자열 리스트로 전처리
for col in ["초등학교", "유치원", "어린이집"]:
    grouped_filtered[col] = grouped_filtered[col].apply(
        lambda x: [f'{name}' for name in x if name] if isinstance(x, list) else []
    )

# 두 데이터 병합
merged = pd.concat([grouped_filtered, target_df], ignore_index=True)


# 행정동코드 기준 병합
final_df = merged.groupby("행정동코드").agg({
    "초등학교": lambda x: sorted(list(set(i for sublist in x for i in sublist))),
    "유치원": lambda x: sorted(list(set(i for sublist in x for i in sublist))),
    "어린이집": lambda x: sorted(list(set(i for sublist in x for i in sublist)))
}).reset_index()

final_df = final_df.rename(columns={"행정동코드": "행자부코드"})

In [119]:
final_df.head()
final_df['행자부코드'].nunique()

428

In [145]:
data = pd.read_excel('./merge/서울시_법정동_행정동_매핑.xlsx')

In [151]:
mapping = [i[:8] for i in data['행정동코드'].astype(str)]

In [125]:
final_df.to_csv('./초등학교_유치원_어린이집_리스트형태.csv', index=False, encoding='utf-8-sig')

In [207]:
df = pd.read_csv('./진짜진짜최수종.csv', encoding='utf-8')

In [209]:
set(mapping) - set(df['행자부코드'].astype(str))

{'11680510'}

In [211]:
df.shape

(425, 36)